In [24]:
import json
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import scipy.misc
import os
import openslide
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import time
from torch.utils.data import Dataset, DataLoader

path = '/scratch/cz2064/myjupyter/BDML/Project/metadata/biospecimen.cart.2020-03-28.json'
with open(path) as f:
    data = json.load(f)

In [2]:
path = '/scratch/cz2064/myjupyter/BDML/Project/metadata/gdc_sample_sheet.2020-03-28.tsv'
df = pd.read_csv(path, sep='\t')
df
# data -> File ID -> File Name

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type
0,e9760244-3db6-48e1-aa57-f2e2b5352934,TCGA-18-3419-01A-01-BS1.258a5fb6-f212-4a6a-a43...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-18-3419,TCGA-18-3419-01A,Primary Tumor
1,c9b69594-7be5-4597-8698-7c2880b14b24,TCGA-56-8305-11A-01-TS1.dc4b057e-a254-4147-84c...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-56-8305,TCGA-56-8305-11A,Solid Tissue Normal
2,79ec2dcc-5ca0-4651-aeac-26db87d4f696,TCGA-34-2600-11A-01-BS1.dd70f786-4ac1-4dfc-b66...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-34-2600,TCGA-34-2600-11A,Solid Tissue Normal
3,70d0010d-8346-4d48-8620-16bbe3523667,TCGA-77-A5G6-01A-01-TS1.694AEB4E-FE41-4226-899...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-77-A5G6,TCGA-77-A5G6-01A,Primary Tumor
4,a369fc28-650b-4587-83dc-78fa8875ef16,TCGA-56-A62T-01A-01-TS1.E3B64735-90EE-436F-901...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-56-A62T,TCGA-56-A62T-01A,Primary Tumor
...,...,...,...,...,...,...,...,...
2076,95900224-50ab-4d5c-9771-8775401206c8,TCGA-77-8145-01A-01-BS1.1068be69-6ef5-40c1-901...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-77-8145,TCGA-77-8145-01A,Primary Tumor
2077,7b8a150b-64d2-4a5c-9992-e941635f4db3,TCGA-39-5037-01A-01-TS1.53e009fc-9769-4121-a10...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-39-5037,TCGA-39-5037-01A,Primary Tumor
2078,a3ad4705-3adb-4e88-b3a7-4b4494632d49,TCGA-96-7544-01A-01-BS1.4c78920d-ef0d-4263-836...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-96-7544,TCGA-96-7544-01A,Primary Tumor
2079,08fdb4d4-10ef-40a2-8a73-ca32ad432ab5,TCGA-63-5131-01A-01-TS1.93fe90a7-97ec-47ab-967...,Biospecimen,Slide Image,TCGA-LUSC,TCGA-63-5131,TCGA-63-5131-01A,Primary Tumor


In [3]:
df.loc[0]['File ID']

'e9760244-3db6-48e1-aa57-f2e2b5352934'

In [4]:
df.loc[0]['File Name']

'TCGA-18-3419-01A-01-BS1.258a5fb6-f212-4a6a-a43a-75541f043038.svs'

In [5]:
df.loc[1]['File ID']

'c9b69594-7be5-4597-8698-7c2880b14b24'

In [6]:
df.loc[1]['File Name']

'TCGA-56-8305-11A-01-TS1.dc4b057e-a254-4147-84ce-4542c61d81ce.svs'

In [7]:
# divide the dataframe into 21 sub_df, to make the preprocess faster
number_of_sub_df = int(len(df)/100)+1
for i in range(number_of_sub_df):
    sub_df = df.loc[i*100:(i+1)*100]
    path = '/scratch/cz2064/myjupyter/BDML/Project/metadata/Preprocess_sub_dataframe/'
    sub_df_file_name = 'df'+ str(i) +'.csv'
    pd.DataFrame.to_csv(sub_df,path+sub_df_file_name)

In [27]:
number = 1
imgage_folder = '/scratch/cz2064/myjupyter/BDML/Project/data/'
df_path = '/scratch/cz2064/myjupyter/BDML/Project/metadata/Preprocess_sub_dataframe/df' + str(number) +'.csv'
df = pd.read_csv(df_path, sep=',')
for i in df.index:
    file_id = df.loc[i,'File ID']
    file_name = df.loc[i,'File Name']
    sample_image = openslide.open_slide(imgage_folder+file_id+'/'+file_name)
    sample_image = np.array(sample_image.get_thumbnail(sample_image.dimensions))
    image_dimensions = sample_image.shape[:2]
    pixels = 1024
    valid_subsamples_count = 0
    valid_subsamples = []
    for i in range(int(image_dimensions[0]/pixels)):
        for j in range(int(image_dimensions[1]/pixels)):
            sub_image_array = sample_image[i*pixels:(i+1)*pixels,j*pixels:(j+1)*pixels,:]
            if white_ratio(sub_image_array) < 0.5:
                subsample_file_name = 'subsample'+ str(valid_subsamples_count)
                save_file = imgage_folder + file_id + '/' + subsample_file_name + '.jpg'
                matplotlib.image.imsave(save_file, sub_image_array)
                valid_subsamples_count = valid_subsamples_count + 1
                valid_subsamples.append(subsample_file_name)
    summary_file = imgage_folder + file_id + '/' + 'subsamples_list.txt'
    f = open(summary_file,"w")
    for i in valid_subsamples:
        i = i + '\n'
        f.write(i)
    f.close()

(36238, 47808, 3)

In [ ]:
df.head()

In [ ]:
# creat subsamples
imgage_folder = '/scratch/cz2064/myjupyter/BDML/Project/data/'
file_id = 'c9b69594-7be5-4597-8698-7c2880b14b24'
file_name = 'TCGA-56-8305-11A-01-TS1.dc4b057e-a254-4147-84ce-4542c61d81ce.svs'
sample_image = openslide.open_slide(imgage_folder+file_id+'/'+file_name)
sample_image = np.array(sample_image.get_thumbnail(sample_image.dimensions))
image_dimensions = sample_image.shape[:2]
pixels = 1024
valid_subsamples_count = 0
valid_subsamples = []
for i in range(int(image_dimensions[0]/pixels)):
    for j in range(int(image_dimensions[1]/pixels)):
        sub_image_array = sample_image[i*pixels:(i+1)*pixels,j*pixels:(j+1)*pixels,:]
        if white_ratio(sub_image_array) < 0.5:
            subsample_file_name = 'subsample'+ str(valid_subsamples_count)
            save_file = imgage_folder + file_id + '/' + subsample_file_name + '.jpg'
            matplotlib.image.imsave(save_file, sub_image_array)
            valid_subsamples_count = valid_subsamples_count + 1
            valid_subsamples.append(subsample_file_name)
summary_file = imgage_folder + file_id + '/' + 'subsamples_list.txt'
f = open(summary_file,"w")
for i in valid_subsamples:
    i = i + '\n'
    f.write(i)
f.close()